In [6]:
import pandas
import aql
conn = aql.Connection("http://arachne.compbio.ohsu.edu")
O = conn.graph("bmeg")

In [7]:
q = O.query().V().where(aql.eq("_label", "Individual"))
q = q.where(aql.and_(aql.eq("source", "tcga"), aql.eq("disease_code", "KIRC")))
q = q.in_("sampleOf").in_("callSetOf").where(aql.eq("method", "MUTECT"))
q = q.in_("variantCall").out("variantIn")
q = q.aggregate(aql.term("agg", "Gene", "symbol"))

res = list(q)
for i in res[0]['agg']['buckets']:
    print i['key'], i['value']


VHL 8
TP53 4
MTOR 3
PBRM1 3
ATE1 2
BRINP3 2
CDH12 2
CSMD3 2
DCC 2
PIK3CA 2
PTEN 2
TTN 2
ABCA7 1
ACSF2 1
ACSM1 1
ACTG1 1
ADAMTS19 1
ADCY9 1
AFF3 1
AHDC1 1
AHNAK2 1
AIFM1 1
ALDH1L2 1
ANKRD13A 1
APBA2 1
ARHGAP44 1
ATCAY 1
ATM 1
ATP2A1 1
B3GAT1 1
B4GALNT4 1
BAK1 1
BIN2 1
BMPR2 1
BRIP1 1
BTBD2 1
C2CD2L 1
C8orf34 1
C9orf84 1
CACNA1C 1
CATSPERD 1
CCDC155 1
CCNG2 1
CD300LG 1
CDH10 1
CDK6 1
CDKL5 1
CEP76 1
CHRM2 1
CLUAP1 1
CNP 1
CNTN1 1
COL11A1 1
COL1A2 1
COL27A1 1
COL5A2 1
COL5A3 1
COPS2 1
CPNE5 1
CRAT 1
CTAGE5 1
CTNNA2 1
CWC27 1
CYP19A1 1
DCDC1 1
DDX53 1
DEFB116 1
DET1 1
DLX6 1
DYNC1I1 1
EFCAB12 1
EHMT2 1
EME2 1
EMX1 1
ENTHD1 1
EPHA8 1
ERMN 1
F13B 1
FAM129B 1
FAM166B 1
FAM209A 1
FAM3A 1
FAM71B 1
FAT4 1
FCRL2 1
FLG2 1
FLRT2 1
FPR3 1
FSTL4 1
GBP6 1
GIMAP8 1
GJA10 1
GP1BA 1
GPHN 1
GPR139 1
GPR179 1
GPR78 1
GPR85 1
GRAMD1A 1
H2AFY2 1
H3F3C 1
HEATR5B 1
HEPHL1 1
HLA-DPA1 1
HMSD 1
HNRNPH1 1
HOXB1 1
HRASLS 1
HSPA4 1
HTT 1
HUNK 1
IGSF11 1
IGSF9B 1
INCENP 1
IQSEC3 1
IRX1 1
IRX6 1
ISL1 1
ITGA10 1
ITIH3 

In [8]:
res

[{u'agg': {u'buckets': [{u'key': u'VHL', u'value': 8},
    {u'key': u'TP53', u'value': 4},
    {u'key': u'MTOR', u'value': 3},
    {u'key': u'PBRM1', u'value': 3},
    {u'key': u'ATE1', u'value': 2},
    {u'key': u'BRINP3', u'value': 2},
    {u'key': u'CDH12', u'value': 2},
    {u'key': u'CSMD3', u'value': 2},
    {u'key': u'DCC', u'value': 2},
    {u'key': u'PIK3CA', u'value': 2},
    {u'key': u'PTEN', u'value': 2},
    {u'key': u'TTN', u'value': 2},
    {u'key': u'ABCA7', u'value': 1},
    {u'key': u'ACSF2', u'value': 1},
    {u'key': u'ACSM1', u'value': 1},
    {u'key': u'ACTG1', u'value': 1},
    {u'key': u'ADAMTS19', u'value': 1},
    {u'key': u'ADCY9', u'value': 1},
    {u'key': u'AFF3', u'value': 1},
    {u'key': u'AHDC1', u'value': 1},
    {u'key': u'AHNAK2', u'value': 1},
    {u'key': u'AIFM1', u'value': 1},
    {u'key': u'ALDH1L2', u'value': 1},
    {u'key': u'ANKRD13A', u'value': 1},
    {u'key': u'APBA2', u'value': 1},
    {u'key': u'ARHGAP44', u'value': 1},
    {u'key': u'